# Installation
Please follow the installation guide for a detailed installation guide,for a quick start you can run the following commands.

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    %pip install --quiet cirq
    import cirq
    print("installed cirq.")
import cirq_google

## Qubits in cirq
Cirq has three main ways of defining qubits:

*   `cirq.NamedQubit`: used to label qubits by an abstract name
*   `cirq.LineQubit`: qubits labelled by number in a linear array 
*   `cirq.GridQubit`: qubits labelled by two numbers in a rectangular lattice.

In [ ]:
# Using named qubits can be useful for abstract algorithms
# as well as algorithms not yet mapped onto hardware.
q0 = cirq.NamedQubit('source')
q1 = cirq.NamedQubit('target')

# Line qubits can be created individually
q3 = cirq.LineQubit(3)

# Or created in a range
# This will create LineQubit(0), LineQubit(1), LineQubit(2)
q0, q1, q2 = cirq.LineQubit.range(3)

# Grid Qubits can also be referenced individually
q4_5 = cirq.GridQubit(4,5)

# Or created in bulk in a square
# This will create 16 qubits from (0,0) to (3,3)
qubits = cirq.GridQubit.square(4)

## Gates And Operations

Cirq has two concepts that are important:

*   A `Gate` is a cirq object that abstracts the unitary transformation matrix.  
*   An `Operation` is a gate applied to a set of qubits (i.e. the multiplication operations betwen a gate and a qubit).

For instance, `cirq.H` is the quantum [Hadamard](https://en.wikipedia.org/wiki/Quantum_logic_gate#Hadamard_(H)_gate) and is a `Gate` object.  `cirq.H(cirq.LineQubit(1))` is an `Operation` object and is the Hadamard gate applied to a specific qubit.

In [ ]:
# Example gates
not_gate = cirq.X
pauli_z = cirq.Z
cnot_gate = cirq.CNOT

# We can extract the unitary matrix from a Gate object as follows
print("Not gate:")
print(cirq.unitary(not_gate))
print("Pauli Z gate:")
print(cirq.unitary(pauli_z))
print("CNOT gate:")
print(cirq.unitary(cnot_gate))

q0, q1 = cirq.LineQubit.range(2)
# Example Operations
not_op = cirq.X(q0)
z_op = cirq.Z(q0)
cnot_op = cirq.CNOT(q0, q1)

# Circuits

The primary representation of quantum programs in Cirq is the `Circuit` class. A `Circuit` is a collection of `Moments`. A `Moment` is a collection of `Operations` that all act during the same abstract time slice. An `Operation` is a some effect that operates on a specific subset of Qubits, the most common type of `Operation` is a `GateOperation`.

The following diagram explains it better:

<img src="./res/CircuitMomentOperation.png" width="500" height="340">



In [ ]:
circuit1 = cirq.Circuit()
# You can create a circuit by appending to it
q0, q1, q2 = cirq.LineQubit.range(3)
circuit1.append(cirq.H(q0))
circuit1.append(cirq.H(q1))
circuit1.append(cirq.H(q2))
# All of the gates are put into the same Moment since none overlap
print("Circutit1:")
print(circuit1)

circuit2 = cirq.Circuit(cirq.H(q0),cirq.CNOT(q1, q2), cirq.H(q2))
# The H gate on q2 will be put in a different moment,
# since it would overlap with CNOT
print("Circuit2:")
print(circuit2)

# Customizable Circuit Layouts
## Defining Moments
Now if we want to change the default behavior we have 2 options:
either to define the moment type objects, or to use an Insert Strategy.
We will first present the first variant because it is more general.

In [ ]:
qubits = cirq.LineQubit.range(3)
cz01 = cirq.CZ(qubits[0], qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1], qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])
circuit = cirq.Circuit((moment0, moment1))

print(circuit)

## Insert Strategy
Insert Strategy offers us the possibility to insert operations where we want by using just a simple argument in the append function.
There are 4 types of them, and they are very well explainde in the following diagram:

<img src="./res/InsertStrategy.png" width="500" height="340">

In [ ]:
q0, q1, q2 = cirq.LineQubit.range(3)

circuit1 = cirq.Circuit()
circuit1.append([cirq.CZ(q0, q1)])
circuit1.append([cirq.H(q0), cirq.H(q2)], strategy=cirq.InsertStrategy.EARLIEST)

circuit2 = cirq.Circuit()
circuit2.append([cirq.H(q0), cirq.H(q1), cirq.H(q2)], strategy=cirq.InsertStrategy.NEW)

circuit3 = cirq.Circuit()
circuit3.append([cirq.CZ(q1, q2)])
circuit3.append([cirq.CZ(q1, q2)])
circuit3.append([cirq.H(q0), cirq.H(q1), cirq.H(q2)], strategy=cirq.InsertStrategy.INLINE)

circuit4 = cirq.Circuit()
circuit4.append([cirq.H(q0)])
circuit4.append([cirq.CZ(q1,q2), cirq.H(q0)], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

print("EARLIEST:")
print(circuit1)
print("NEW:")
print(circuit2)
print("INLINE:")
print(circuit3)
print("NEW_THEN_INLINE:")
print(circuit4)

# Simulation

The results of the application of a quantum circuit can be calculated by a `Simulator`.  Cirq comes bundled with a simulator that can calculate the results of circuits up to about a limit of 20 qubits.  It can be initialized with `cirq.Simulator()`.

There are two different approaches to using a simulator:

*   `simulate()`:  Since we are classically simulating a circuit, a simulator can directly access and view the resulting wave function.  This is useful for debugging, learning, and understanding how circuits will function.  
*   `run()`:  When using actual quantum devices, we can only access the end result of a computation and must sample the results to get a distribution of results.  Running the simulator as a sampler mimics this behavior and only returns bit strings as output.

Let's try to simulate a 2-qubit "Bell State":

In [ ]:
bell_circuit = cirq.Circuit()
q0, q1 = cirq.LineQubit.range(2)
bell_circuit.append(cirq.H(q0))
bell_circuit.append(cirq.CNOT(q0,q1))
print("Bell Circuit:")
print(bell_circuit)

# Initialize Simulator
simulator=cirq.Simulator()

print("------------------- Simulate the circuit ----------------------")
results=simulator.simulate(bell_circuit)
print(results)
print("---------------------------------------------------------------")

# For sampling, we need to add a measurement at the end
bell_circuit.append(cirq.measure(q0, q1, key='result'))

print("-----------------------Sample the circuit---------------------")
samples=simulator.run(bell_circuit, repetitions=1000)
# Print a histogram of results
print(samples.histogram(key="result"))
print("--------------------------------------------------------------")

# Basic Exercises
## 1. Create a Circuit
Create the following circuit using moments or Insert strategy
```
    0:---H------@------
                |
    1:------H---X---@--
                    |
    2:------H---Z---X--
```

In [ ]:
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit()


# TODO: create a moment for each vertical line in the circuit
# above and add the operations to their moment. Alternatively
# you can use Insert Strategy (HINT: NEW_THEN_INLINE)
print(circuit)
print()

## 2. Quantum Adder 
Create a simple quantum circuit that will simulate a full adder, start from the following picture and follow the TODOs:

<img src="./res/Quantum_Full_Adder.png" width="500" height="340">

In [ ]:
def create_circuit(circuit, a, b, s, c_out):
    # TODO: ADD TOFFOLI
    # TODO: ADD CNOT
    # TODO: ADD TOFFOLI
    # TODO: ADD CNOT
    # TODO: ADD CNOT
    # TODO: ADD measurements on s and c_out. Use key="results" for measure method
    #       Put The mesurments in a different moment than the last CNOT gate
    print(circuit)
    print()

a, b, s, c_out = cirq.LineQubit.range(4)
circuit = cirq.Circuit()
create_circuit(circuit, a, b, s, c_out)

# 3. Run the circuit
   Follow the TODOs to construct the 4 inputs (i.e. a=0, b=0; a=0, b=1; a=1, b=0; a=1, b=1) and run the circuits.

In [ ]:
for x in [0,1]:
    for y in [0,1]:
        # Create a new circuit
        a, b, s, c_out = cirq.LineQubit.range(4)
        circuit = cirq.Circuit()
        simulator = cirq.Simulator()
        print("A = " + str(x) + " B = " + str(y))
        if x == 1:
            # TODO: add a NOT gate for A. Remove None
            None
        if y == 1:
            # TODO: add a NOT gate for B. Remove None
            None
        # create the circuit
        create_circuit(circuit, a, b, s, c_out)
        # TODO: use simulator.run() to sample the circuit. Store the return value in `result` !!
        # TODO: uncoment the following line after solving the previous TODO
        # print("The result is [S, C_OUT] = ", result.measurements["results"][0])
        print("______________________________________________________________")